# Divide and Conquer 
***

### The two deceiving classifications
- **Multi Threading** 
- **Multi Processing** 

### What is  Multithreading?
- **A single core(processor) in your computer can mimic multi(N)processor, This will allow you to run multiple(N) programs concurrently(not exactly!).**
- **Multiple processor can share same memory space. So that a single variable can be updated or used by multiple processor at  the same time.**

### What is *Multiprocessing*?
- **A single core(processor) is always single. So for running program concurrently(exactly), you need to use multiple physical cores(processors)**
- **Multipleprocessor doesn't share memory. So, we need to make them talk among themselves.**

### High Perfomance Computing(HPC)
- **Uses both of this concept efficiently** 
***

# Message Passing Interface(MPI)

### The Big MPI is mainly used in C, C++ and Fortran

### But however BIG it is ... python has a very stretchy Belly!
***

# Environment setup

Minimum requirements:
- MPICH Library + OpenMPI Library
- Python > 2.7.5
- mpi4py Python module

Installisation:
- MPICH/OpenMPI
  ```
  sudo apt-get install aptitude
  sudo aptitude install mpich mpich2 openmpi
  ```
  You can search and install particular library and softwares using 
  ```
  sudo aptitude search <lib>
  ```
  
- Python
   ```
   sudo aptitude install python
   sudo aptitude install python-pip
   pip install mpi4py
   ```
   or,
   ```
   sudo aptitude install python-mpi4py
   ```
   ***

# Let's make you computer's cores to communicate themselves

### Import Library

In [ ]:
from mpi4py import MPI

### Mandatory Communication intialization 

In [ ]:
# Meduim of communication
comm = MPI.COMM_WORLD
# Name of the Communicator
rank = comm.Get_rank()
# Total no. of Communicators
size = comm.Get_size()

## 3 arguments for communication

### (source=0, dest=1, tag=1001)

### source = From where you are communicating?

### dest = To whom you are communicating?

### tag = Are you communicating to me or  to someone else?
***

# Let's Script

## Introduction.py
A python script that prints from every processor

run using cmd:
```
mpiexec -n 2 python Introduction.py
```

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

print('Hello world from Processor %d' % rank)

***
# Point to Point Communication

## PPC_1.py
Sending and Receiving python pickle

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    comm.send(data, dest=1, tag=11)
elif rank == 1:
    data = None
    print('Before receving: ', data)
    data = comm.recv(source=0, tag=11)
    print('After receving: ', data)

## PPC_2.py
Sending and Receiving python pickle without blocking execution

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    req = comm.isend(data, dest=1, tag=11)
    req.wait()
elif rank == 1:
    data = None
    print('Before receving: ', data)
    req = comm.irecv(source=0, tag=11)
    print('After receving: ', data)
    data = req.wait()
    print('After checking:', data)

## PPC_3.py
Sending and Receiving Numpy objects

In [ ]:
from mpi4py import MPI
import numpy

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# passing MPI datatypes explicitly
if rank == 0:
    data = numpy.arange(1000, dtype='i')
    comm.Send([data, MPI.INT], dest=1, tag=77)
elif rank == 1:
    data = numpy.empty(1000, dtype='i')
    comm.Recv([data, MPI.INT], source=0, tag=77)

# automatic MPI datatype discovery
if rank == 0:
    data = numpy.arange(100, dtype=numpy.float64)
    comm.Send(data, dest=1, tag=13)
elif rank == 1:
    data = numpy.empty(100, dtype=numpy.float64)
    comm.Recv(data, source=0, tag=13)

***
# Collective Communication

## CC_1.py
Broadcasting a Python pickle

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'key1' : [7, 2.72, 2+3j],
            'key2' : ( 'abc', 'xyz')}
else:
    data = None
data = comm.bcast(data, root=0)

if rank != 0:
    print('After broadcasting', data)

## CC_2.py
Scattering a Python pickle

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = [(i+1)**2 for i in range(size)]
else:
    data = None
data = comm.scatter(data, root=0)
print('From Processor %d :' data)

## CC_3.py
Gathering a Python pickle

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

data = (rank+1)**2
data = comm.gather(data, root=0)
if rank == 0:
    print(data)

# Writing for loops with MPI

## forloop.py
Execute it  using 

`time python forloop.py`

In [ ]:
no = 100000000
sum = 0
for i in range(1,no+1):
    sum += i
print(sum)

## forloop_MPI.py
Execute it  using 

`time mpiexec -n 2 python forloop_MPI.py`

In [ ]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

no = 100000000
summ = 0

for i in range(rank,no+1,size):
        summ += i
tot = comm.gather(summ,root=0)

if rank==0:
   print(sum(tot))